In [49]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import imblearn
import warnings
warnings.filterwarnings('ignore')

In [50]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [51]:
# cd drive/MyDrive/NN

In [52]:
def upsample_label(temp_x, temp_y):
    y_time_list = temp_y['time'].values
    y_label = temp_y['label'].values
    jdx = 0
    label_list = []
    for index,row in temp_x.iterrows():
        try:
            if row['time'] > y_time_list[jdx]:
                jdx+=1
            label_list.append(y_label[jdx])
        except:
            label_list.append(y_label[jdx - 1])
    return label_list

In [53]:
filepath = 'TrainingData/'
column_list = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z','subject', 'time', 'label']
final_df = pd.DataFrame([], columns = column_list)
for i in glob.glob(filepath + '*.csv'):
    df_type = i.split('.')[0].split('__')[1]
    file_name = i.split("\\")[1].split("__")[0]
#     file_name = i.split('/')[1].split('__')[0]
    if df_type == 'x':
        x = open(filepath + file_name + '__x.csv')
        x = pd.read_csv(x)
        x['subject'] = [file_name] * x.shape[0]
        
        x_time = open(filepath + file_name + '__x_time.csv')
        x_time = pd.read_csv(x_time)
        x['time'] = x_time
        
        y = open(filepath + file_name + '__y.csv')
        y = pd.read_csv(y)
        y['subject'] = [file_name] * y.shape[0]
        
        y_time = open(filepath + file_name + '__y_time.csv')
        y_time = pd.read_csv(y_time)
        y['time'] = y_time
        
        y.columns = ['label', 'subject', 'time']
        label_list = upsample_label(x, y)
        x['label'] = label_list
        x.columns = column_list
        final_df = pd.concat([final_df, x], ignore_index=True)

In [54]:
from scipy import stats

def create_windows(X, y, time_steps=1, step=1):
    Xs, ys = [], []
      
    for i in range(0, len(X) - time_steps + 1, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])

    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [55]:
X = final_df[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = final_df['label']

TIME_STEPS = 40
STEP = 1

X_data, y_data = create_windows(X, y, time_steps=TIME_STEPS,step=STEP)

In [56]:
print(X.shape, X_data.shape)
print(y.shape, y_data.shape)

(1341617, 6) (1341578, 40, 6)
(1341617,) (1341578, 1)


In [57]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

df_label = pd.DataFrame(y_data, columns=['label'])
df_label['index_'] = df_label.index

## Library for performing undersampling
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=1)
df_balanced, balanced_labels = rus.fit_resample(df_label, df_label['label'])

sos = SMOTE(sampling_strategy='not majority')
df_balanced, balanced_labels  = sos.fit_resample(df_balanced, balanced_labels)

df_balanced = pd.DataFrame(df_balanced, columns=['label', 'index_'])

X_data = X_data[df_balanced['index_'].values]
y_data = y_data[df_balanced['index_'].values]

In [58]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

## Converting label to OneHot Encoding
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(y_train)
y_train = enc.transform(y_train)
y_val = enc.transform(y_val)

In [59]:
X_train = np.asarray(X_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')

In [60]:
print(X_val.shape, y_val.shape)

(65980, 40, 6) (65980, 4)


In [61]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(40, 6)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy')

In [62]:
cnn = model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
4811/4811 [==============================] - 24s 5ms/step - loss: 0.3021
Epoch 2/50
4811/4811 [==============================] - 23s 5ms/step - loss: 0.1800
Epoch 3/50
4811/4811 [==============================] - 23s 5ms/step - loss: 0.1486
Epoch 4/50
4811/4811 [==============================] - 24s 5ms/step - loss: 0.1288
Epoch 5/50
4811/4811 [==============================] - 23s 5ms/step - loss: 0.1157
Epoch 6/50
4811/4811 [==============================] - 24s 5ms/step - loss: 0.1070
Epoch 7/50
4811/4811 [==============================] - 23s 5ms/step - loss: 0.1000
Epoch 8/50
4811/4811 [==============================] - 25s 5ms/step - loss: 0.0937
Epoch 9/50
4811/4811 [==============================] - 25s 5ms/step - loss: 0.0898
Epoch 10/50
4811/4811 [==============================] - 25s 5ms/step - loss: 0.0851
Epoch 11/50
4811/4811 [==============================] - 24s 5ms/step - loss: 0.0820
Epoch 12/50
4811/4811 [==============================] - 25s 5ms/step - lo

In [63]:
accuracy = model.evaluate(X_val, y_val, verbose=1)

2062/2062 [==============================] - 6s 3ms/step - loss: 0.0615


In [64]:
y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis = 1)
y_val = np.argmax(y_val, axis = 1)

In [65]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     16405
           1       0.99      1.00      1.00     16470
           2       0.99      1.00      0.99     16422
           3       0.98      0.96      0.97     16683

    accuracy                           0.98     65980
   macro avg       0.98      0.98      0.98     65980
weighted avg       0.98      0.98      0.98     65980



In [68]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 38, 32)            608       
                                                                 
 conv1d_13 (Conv1D)          (None, 36, 64)            6208      
                                                                 
 conv1d_14 (Conv1D)          (None, 34, 64)            12352     
                                                                 
 conv1d_15 (Conv1D)          (None, 32, 32)            6176      
                                                                 
 dropout_4 (Dropout)         (None, 32, 32)            0         
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 16, 32)           0         
 1D)                                                             
                                                      

In [69]:
def create_test_windows(X, time_steps=1, step=1):
    Xs = []
    for i in range(0, len(X) - time_steps + 1, step):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

In [70]:
def test_windows(X, time_steps, step):
    Xs = []
    n_records = X.shape[0]
    remainder = (n_records - time_steps) % step
    num_windows = 1 + int((n_records - time_steps - remainder) / step)
    for k in range(num_windows):
        Xs.append(X[k*step:time_steps-1+k*step+1])
    return np.array(Xs)

In [71]:
filepath = 'TestData/'
column_list = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z','subject', 'time']
columns = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
df_test_data = pd.DataFrame([], columns = column_list)

## Reading prediction file 
for idx in glob.glob(filepath + '*.csv'):
    df_type = idx.split('.')[0].split('__')[1]

    if df_type == 'x':
        subject_name = idx.split('.')[0].split('__')[0].split('/')[-1]
        x_path = idx.split('.')[0].split('__')[0] + '__' + df_type + '.csv'
        x_time_path = idx.split('.')[0].split('__')[0] + '__' + 'x_time' + '.csv'
        y_time_path = idx.split('.')[0].split('__')[0] + '__' + 'y_time' + '.csv'
        df_x = pd.read_csv(x_path, names=column_list[:6])

        print("Original number of values: ", df_x.shape)
        ## Converting windows for test data 
        sampled_test = create_test_windows(df_x[columns],TIME_STEPS, STEP)
        print("Number of windows: ", len(sampled_test))

        ## Making inference on test data 
        pred_test = model.predict(sampled_test)
        pred_test = np.argmax(pred_test, axis=1)
        pred_test = list(pred_test)
        pred_test = np.asarray(pred_test)
        pred_test = np.concatenate([pred_test, np.asarray([pred_test[-1]]*120)])
        pred_test = pd.DataFrame(pred_test, columns=['label'])

        ## Sampling the prediction to match the test set sampling frequency. 
        df_mode = pred_test['label'].rolling(window=4, min_periods=1).apply(lambda x: int(stats.mode(x)[0]))[::4]

        y_time = pd.read_csv(y_time_path, header=None).shape[0]
        print(y_time, len(df_mode.values))

        pred_df = pd.DataFrame(df_mode.values)
        pred_df = pred_df.astype('int64', copy=False)
        pred_df.to_csv(subject_name + '__y.csv', index=False, columns=None)

Original number of values:  (37991, 6)
Number of windows:  37952
9498 9518
Original number of values:  (49081, 6)
Number of windows:  49042
12270 12291
Original number of values:  (51761, 6)
Number of windows:  51722
12940 12961
Original number of values:  (45319, 6)
Number of windows:  45280
11330 11350
